### Importing the data and the functions

In [ ]:
include("importData.jl")
include("DDEmodel.jl")
include("plot.jl")

# import data from the path

# population data, g2 data, g1 data, initial number of cells in g1, initial number of cells in g2
# lapatinib
pop_l, g2_l, g1_l, g1_0_l, g2_0_l = import_data(joinpath("..", "data", "lap.csv"),
        joinpath("..", "data", "lap_pop.csv"));

# doxorubicin
pop_d, g2_d, g1_d, g1_0_d, g2_0_d = import_data(joinpath("..", "data", "dox.csv"),
        joinpath("..", "data", "dox_pop.csv"));

# gemcitabine
pop_g, g2_g, g1_g, g1_0_g, g2_0_g = import_data(joinpath("..", "data", "gem.csv"),
        joinpath("..", "data", "gem_pop.csv"));

# paclitaxel
pop_t, g2_t, g1_t, g1_0_t, g2_0_t = import_data(joinpath("..", "data", "taxol1.csv"),
        joinpath("..", "data", "taxol1_pop.csv"));

### A) cartoon 

In [ ]:
# will do in the Affinity designer of power point

### B) ODE fitting of Lapatinib data

### F) DDE fitting

### G) DDE parameters

### H) DDE Hill fitting 

### I) DDE Hill Parameters